# Credit Card Fraud Detection
The datasets contains transactions made by credit cards in September 2013 by european cardholders. 
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. 
The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. 
Due to data confidentiality, original features are unknown. 
Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. 
Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. 
The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 
Feature 'Class' is the target variable and it takes value 1 in case of fraud and 0 otherwise.


## Imports

In [1]:
import ibm_db
import ibm_db_dbi

import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

from time import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, auc, roc_curve, recall_score, classification_report


import pickle
from joblib import dump, load

In [2]:
run db2.ipynb

Db2 Extensions Loaded.


In [3]:
%sql connect to bludb user db2inst1 using redhat host 192.168.154.129 port 31382

Connection successful.


In [41]:
%sql drop table credit_card

Command completed.


In [42]:
%%sql
 create table CREDIT_CARD ( 
    "Time" Decfloat, 
     "V1"  Decfloat,
     "V2"  Decfloat,
     "V3" Decfloat,
     "V4" Decfloat,
     "V5" Decfloat,
     "V6" Decfloat,
     "V7" Decfloat,
     "V8" Decfloat,
     "V9" Decfloat,
     "V10" Decfloat,
     "V11" Decfloat,
     "V12" Decfloat,
     "V13" Decfloat,
     "V14" Decfloat,
     "V15" Decfloat,
     "V16" Decfloat,
     "V17" Decfloat,
     "V18" Decfloat,
     "V19" Decfloat,
     "V20" Decfloat,
     "V21" Decfloat,
     "V22" Decfloat,
     "V23" Decfloat,
     "V24" Decfloat,
     "V25" Decfloat,
     "V26" Decfloat,
     "V27" Decfloat,
     "V28" Decfloat,
     "Amount" Decimal(20),
     "Class" Varchar(16))

Command completed.


In [33]:
db2inst1_directory = '/mnt/blumeta0/scratch'
admin_cmd = 'UPDATE DB CONFIG FOR BLUDB USING EXTBL_LOCATION "' + db2inst1_directory + '" IMMEDIATE'

In [34]:
%sql CALL ADMIN_CMD({admin_cmd})

In [43]:
%sql insert into credit_card SELECT * FROM EXTERNAL '/mnt/blumeta0/scratch/creditcard.csv' like CREDIT_CARD USING(DELIMITER ',' SKIPROWS 1 ) 

Command completed.


In [44]:
%sql select count(*) from credit_card

,1
0,284807.0


## Connect to Db2 Instance

In [4]:
conn_str = "DATABASE=CRCARD;" + \
           "HOSTNAME=entb06.canlab.ibm.com;"+ \
           "PROTOCOL=TCPIP;"  + \
           "PORT=50000;" + \
           "UID=PERFPOL2;" + \
           "PWD=blu4speed;" 


ibm_db_conn = ibm_db.connect(conn_str,"","")
conn = ibm_db_dbi.Connection(ibm_db_conn)
print('Connection to Db2 Instance Created!')

Connection to Db2 Instance Created!


## Pull data from Db2

In [4]:

df = %sql select * from credit_card

In [7]:
print(df.head())

     Time                 V1                  V2                  V3  \
0  166198  -35.5485392112513   -31.8504841430453   -48.3255893623954   
1  166209   1.91383255057816   -1.85383655093476   -2.58959721102643   
2  166213   2.26848214210092   -1.40445396569177  -0.182097641756727   
3  166216   2.01055983289163   0.080389943350253   -1.61392048903693   
4  166219   1.81139876779266  -0.216305351890302   -1.83964430925716   

                  V4                  V5                  V6  \
0   15.3041839851875   -113.743306711146    73.3016255459646   
1  -1.62113572438793  -0.289451333068947  -0.620987697616247   
2   -1.4859860000569   -1.71796541134622  -0.613119031573485   
3  0.351637530758496   0.354545281147957  -0.668284415220535   
4  0.338289635979055   0.309310405370954  -0.488561472685394   

                   V7                   V8                  V9  \
0    120.589493945238    -27.3473599337598   -3.87242453038611   
1  0.0558874868475418   -0.507179751382387   -2.15

## Data Exploration

In [5]:
print(df.head())
print('-'*40)
print(df.info())
print('-'*40)
print(df.isnull().sum()) # --> no null values!
print('-'*40)

     Time                 V1                  V2                  V3  \
0  166198  -35.5485392112513   -31.8504841430453   -48.3255893623954   
1  166209   1.91383255057816   -1.85383655093476   -2.58959721102643   
2  166213   2.26848214210092   -1.40445396569177  -0.182097641756727   
3  166216   2.01055983289163   0.080389943350253   -1.61392048903693   
4  166219   1.81139876779266  -0.216305351890302   -1.83964430925716   

                  V4                  V5                  V6  \
0   15.3041839851875   -113.743306711146    73.3016255459646   
1  -1.62113572438793  -0.289451333068947  -0.620987697616247   
2   -1.4859860000569   -1.71796541134622  -0.613119031573485   
3  0.351637530758496   0.354545281147957  -0.668284415220535   
4  0.338289635979055   0.309310405370954  -0.488561472685394   

                   V7                   V8                  V9  \
0    120.589493945238    -27.3473599337598   -3.87242453038611   
1  0.0558874868475418   -0.507179751382387   -2.15

## Data Transformation

In [5]:
# remove redundant feature
df = df.drop('Time',axis=1)

# create feature and target set
X = df.drop('Class',axis=1)
y = df['Class']
    

# split data before applying transformations
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.36,random_state=42, stratify=y)



# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(y_train, return_counts=True)
test_unique_label, test_counts_label = np.unique(y_test, return_counts=True)
print('-' * 100)

print('Label Distributions: \n 0   1')
print('Training Set: ', train_counts_label/ len(y_train))
print('Testing Set: ', test_counts_label/ len(y_test))

----------------------------------------------------------------------------------------------------
Label Distributions: 
 0   1
Training Set:  [0.99827185 0.00172815]
Testing Set:  [0.99827369 0.00172631]


In [6]:
print('Full Dataset shape: ', df.shape)
print('Training Features shape: ',X_train.shape)
print('Training Labels shape: ',y_train.shape)
print('Testing Features shape: ',X_test.shape)
print('Testing Labels shape: ',y_test.shape)

Full Dataset shape:  (284807, 30)
Training Features shape:  (182276, 29)
Training Labels shape:  (182276,)
Testing Features shape:  (102531, 29)
Testing Labels shape:  (102531,)


In [8]:
# 'Amount' feature needs to be normalized for both training and test set
scaler = StandardScaler()
X_train['AMOUNT_SCALED'] = scaler.fit_transform(X_train['Amount'].values.reshape(-1,1))
X_train = X_train.drop('Amount',axis=1)

X_test['AMOUNT_SCALED'] = scaler.transform(X_test['Amount'].values.reshape(-1,1))
X_test=X_test.drop('Amount',axis=1)

# use oversampling to add more instances of 'fraud' class for training
sm = SMOTE(random_state=42)

X_train_os, y_train_os = sm.fit_sample(X_train,y_train)

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(y_train_os, return_counts=True)
test_unique_label, test_counts_label = np.unique(y_test, return_counts=True)
print('-' * 100)

print('New Label Distributions: \n 0   1 \n')
print('Training Set (Over-Sampled): ', train_counts_label/ len(y_train_os))
print('Testing Set (Not Over-Sampled): ', test_counts_label/ len(y_test))

----------------------------------------------------------------------------------------------------
New Label Distributions: 
 0   1 

Training Set (Over-Sampled):  [0.5 0.5]
Testing Set (Not Over-Sampled):  [0.99827369 0.00172631]


## Copy Testing Data to CREDIT_CARD_PREDICTION Table

In [10]:
# Clear the table
sql = "DELETE FROM CC_PREDICT_SCALED" #"DELETE FROM CREDIT_CARD_PREDICTION"
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)
print("Number of affected rows: ", ibm_db.num_rows(stmt))

Number of affected rows:  0


In [11]:
# Confirm table is empty
sql = 'SELECT * FROM CC_PREDICT_SCALED'
df_test = pd.read_sql(sql,conn)
print('Successfully pulled test data from Db2!')
df_test.head()

Successfully pulled test data from Db2!


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,AMOUNT


In [12]:
# Insert X_test into CREDIT_CARD_PREDICTION table
# put X_test data into tuples
tuple_of_tuples = tuple([tuple(x) for x in X_test.values])

sql = "INSERT INTO CC_PREDICT_SCALED VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"

stmt = ibm_db.prepare(ibm_db_conn, sql)

ibm_db.execute_many(stmt, tuple_of_tuples)
row_count = ibm_db.num_rows(stmt)
print("inserted {} rows".format(row_count))

inserted 102531 rows


In [13]:
# Confirm table is populated properly
sql = 'SELECT * FROM CC_PREDICT_SCALED'
df_test = pd.read_sql(sql,conn)
print('Successfully pulled test data from Db2!')
df_test.head(10)

Successfully pulled test data from Db2!


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,AMOUNT
0,-0.905252,1.232731,0.683559,0.012652,1.358511,-0.254129,1.898066,-0.380180,-0.904601,-0.076524,...,0.100210,-0.002120,0.328463,-0.698855,-0.376579,1.307757,-0.374461,-0.027436,-0.135352,-0.150152
1,-1.847908,2.119865,-0.875599,-1.078042,-0.016920,-0.767172,0.531382,0.444832,0.845560,1.587990,...,0.657388,-0.434762,-0.635526,0.109258,-0.370988,-0.177327,0.115749,0.286479,-0.094258,-0.313372
2,-0.140836,-0.303637,2.294315,-1.422036,-1.387915,0.279619,-1.070834,-0.587764,-0.896455,0.144522,...,-0.468192,0.626363,-0.314809,-0.300712,0.091875,1.106761,0.812381,0.062791,0.143315,-0.175354
3,-4.198735,0.194121,-3.917586,3.920748,-1.875486,-2.118933,-3.614445,1.687884,-2.189871,-4.684233,...,1.003350,0.801312,-0.183001,-0.440387,0.292539,-0.144967,-0.251744,1.249414,-0.131525,0.594925
4,-0.934305,1.394167,1.707510,-0.182922,-0.059974,-1.081418,1.078031,-0.498115,0.440596,1.036788,...,0.609200,-0.401842,-0.543589,-0.033801,0.683571,-0.134997,0.007814,0.257391,-0.133496,-0.329054
5,-0.281005,1.195619,-0.106102,-0.590382,0.266178,-1.074682,0.714535,0.169922,-0.361994,-0.663133,...,-0.078900,-0.216640,-0.522549,0.090039,-0.088899,-0.410126,0.146386,0.129220,0.035133,-0.340944
6,-0.266383,0.394598,1.332579,-0.631239,0.299064,-0.644056,0.779659,-0.619076,0.916360,-0.052682,...,-0.178343,-0.150175,0.016261,0.195920,-0.056361,-0.704178,-0.764693,-0.552502,-0.425813,-0.323011
7,1.183120,0.408614,0.740999,2.580974,-0.144832,0.081847,-0.061532,0.038867,-0.214730,0.528982,...,-0.163165,-0.323377,-0.835170,0.024106,-0.152089,0.433805,-0.148282,0.006526,0.021541,-0.318784
8,1.245674,0.166975,0.488306,0.635322,-0.562777,-1.011073,0.014953,-0.160211,0.170362,-0.044575,...,-0.132080,-0.262581,-0.816264,0.140304,0.357827,0.186423,0.096544,-0.035866,0.018495,-0.313254
9,-3.005142,3.245751,-1.628285,-1.090845,-0.108972,-1.185605,0.744452,0.136761,2.379494,3.596699,...,1.508013,-0.038671,1.137997,0.058913,1.070088,-0.263812,-0.299221,1.028083,0.270267,-0.333913


## Model Selection and Training

As the objective is to detect fraud, the evaluation metric will be Recall not Accuracy!

When designing the model, Decision Tree Classifier was found to be the best, so training and evaluating the Logistic Regression Model and Stochastic Gradient Descent Model will be skipped for the purposes of this experiment

In [ ]:
# Function for creating confusion matrix plot
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14, title=''):
    df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names)
    fig = plt.figure(figsize=figsize)
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title(title)
    plt.show()

In [ ]:
# Function for plotting learning curves

def plot_learning_curve(model,X_train,y_train,title,scoring=None):
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train,train_sizes=np.linspace(0.1, 1.0, 10), scoring=scoring) #, cv=ShuffleSplit(n_splits=100, test_size=0.2, random_state=42))
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="#ff9124")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="#ff9124", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="#2492ff", label="Cross-validation score")
    plt.title(title, fontsize=14)
    plt.xlabel('Training size (m)')
    plt.ylabel('Score')
    plt.grid(True)
    plt.legend(loc="best")
    plt.show()

In [ ]:
# Decision Tree Classifier
dtc = DecisionTreeClassifier()
t0 = time()
mod1 = dtc.fit(X_train_os,y_train_os)
print('Decision Tree Classifier Training Time: ', round(time()-t0, 3), "s")
print('-'*40)
print('Decision Tree Classifier Training Accuracy: ', mod1.score(X_train_os,y_train_os))
print('-'*40)
print('Decision Tree Classifier Training Classification Report \n', classification_report(y_train_os, mod1.predict(X_train_os)))
print('-'*40)
dtc_train_cm = confusion_matrix(y_train_os, mod1.predict(X_train_os))
print_confusion_matrix(dtc_train_cm, ['Not Fraud', 'Fraud'], title='DTC Training Conf Mtx')
#plot_learning_curve(mod1,X_train_os,y_train_os,"Decision Tree Classifier Learning Curve")

In [ ]:
# # Logistic Regression
# logreg = LogisticRegression()
# t0=time()
# mod2 = logreg.fit(X_train_os,y_train_os)
# print('Logistic Regression Training Time: ', round(time()-t0, 3), "s")
# print('-'*40)
# print('Logistic Regression Training Accuracy: ', mod2.score(X_train_os,y_train_os))
# print('-'*40)
# print('Logistic Regression Training Classification Report \n', classification_report(y_train_os, mod2.predict(X_train_os)))
# print('-'*40)
# lr_train_cm = confusion_matrix(y_train_os, mod2.predict(X_train_os))
# print_confusion_matrix(lr_train_cm, ['Not Fraud', 'Fraud'], title='LR Training Conf Mtx')
# #plot_learning_curve(mod2,X_train_os,y_train_os,"Logisitic Regression Learning Curve")

In [ ]:
# # Stochastic Gradient Descent Classifier
# sgdc = SGDClassifier()
# t0=time()
# mod3 = sgdc.fit(X_train_os,y_train_os)
# print('Support Vector Classifier Training Time: ', round(time()-t0, 3), "s")
# print('-'*40)
# print('SGD Classifier Training Accuracy: ', mod3.score(X_train_os,y_train_os))
# print('-'*40)
# print('SGC Classifier Training Classification Report \n', classification_report(y_train_os, mod3.predict(X_train_os)))
# print('-'*40)
# sgdc_train_cm = confusion_matrix(y_train_os, mod3.predict(X_train_os))
# print_confusion_matrix(sgdc_train_cm, ['Not Fraud', 'Fraud'], title='SGDC Training Conf Mtx')
# #plot_learning_curve(mod3,X_train_os,y_train_os,"SGD Learning Curve")

## Tune Model

Select Decision Tree classifier for tuning

In [ ]:
params = {
    "criterion": ["gini", "entropy"], 
    "max_depth": list(range(1,4,1)), 
    "min_samples_leaf": list(range(1,7,1))
}

# use grid search cross validation to find best model
grid_search = GridSearchCV(mod1,params,cv=5,n_jobs=3, verbose=5, scoring='recall') 
grid_search.fit(X_train_os,y_train_os)

#Choose the best fitting model
mod_best = grid_search.best_estimator_ 
print('Final Model has parameters: ', grid_search.best_params_)
print('-'*40)
#Train best model on training data
mod_best.fit(X_train_os,y_train_os)

print('Final Model Training Accuracy: ', mod_best.score(X_train_os,y_train_os))
print('-'*40)
print('Best Model Training Classification Report: \n', classification_report(y_train_os, mod_best.predict(X_train_os)))
print('-'*40)
best_train_cm = confusion_matrix(y_train_os, mod_best.predict(X_train_os))
print_confusion_matrix(best_train_cm, ['No Fraud', 'Fraud'], title='Best Model Training Conf Mtx')

In [ ]:
plot_learning_curve(mod_best,X_train_os,y_train_os,"Best Model Learning Curve")
plot_learning_curve(mod_best,X_train_os,y_train_os,"Best Model Learning Curve (Recall)",scoring='recall')

## Model Evaluation

In [ ]:
# Make prediction
y_pred = mod_best.predict(X_test)
print('Final Model Testing Accuracy: ', mod_best.score(X_test,y_test))
print('-'*40)
print('Final Model Testing Classification Report \n', classification_report(y_test, y_pred))
print('-'*40)

# Plot confusion matrix
final_cm = confusion_matrix(y_test,y_pred)
print_confusion_matrix(final_cm, ['No Fraud', 'Fraud'], title='Final Model Testing Confusion Matrix')

## Save Model and Scaler

In [ ]:
# Store model
dump(mod_best, 'test/saved_model.joblib') #/data2/home/apu/saved_model.joblib
print('Model Saved!')
# Store scaler
dump(scaler, 'test/saved_scaler.joblib') #/data2/home/apu/saved_scaler.joblib
print('Scaler Saved!')